In [2]:
import netCDF4 as nc
import pandas as pd

How to process nc dataset https://freedium.cfd/https://towardsdatascience.com/read-netcdf-data-with-python-901f7ff61648 

In [24]:
# Open the NetCDF file
dataset = nc.Dataset('pnwrain.50km.daily.4994.nc')

In [25]:
print(dataset)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): lat(17), lon(16), time(16801)
    variables(dimensions): float32 lat(lat), float32 lon(lon), float64 time(time), int16 data(time, lat, lon)
    groups: 


In [26]:
# Convert the dataset to a DataFrame
df = pd.DataFrame()

https://www.youtube.com/watch?v=hrm5RmsVXo0&list=PLLxyyob7YmEE8S3QDs1PZQkiBxA4zn_Gx&index=3 

Dimension

In [27]:
for dim in dataset.dimensions.values():
    print(dim)

<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 17
<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 16
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 16801


In [28]:
lat = dataset.variables['lat']

print(lat)

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    title: Latitude
    units: degrees_north
    scale_factor: 1.0
    add_offset: 0.0
unlimited dimensions: 
current shape = (17,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [30]:
lon = dataset.variables['lon']

print(lon)

<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    title: Longitude
    units: degrees_east
    scale_factor: 1.0
    add_offset: 0.0
unlimited dimensions: 
current shape = (16,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [29]:
time = dataset.variables['time']

print(time)

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    title: Time
    units: days    since 1949- 1- 1  0: 0: 0
    scale_factor: 1.0
    add_offset: 0.0
unlimited dimensions: time
current shape = (16801,)
filling on, default _FillValue of 9.969209968386869e+36 used


Variable Metadata

In [7]:
for var in dataset.variables.values():
    print(var)

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    title: Latitude
    units: degrees_north
    scale_factor: 1.0
    add_offset: 0.0
unlimited dimensions: 
current shape = (17,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    title: Longitude
    units: degrees_east
    scale_factor: 1.0
    add_offset: 0.0
unlimited dimensions: 
current shape = (16,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    title: Time
    units: days    since 1949- 1- 1  0: 0: 0
    scale_factor: 1.0
    add_offset: 0.0
unlimited dimensions: time
current shape = (16801,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
int16 data(time, lat, lon)
    long_name: mm/day
    add_offset: 0.0
    scale_factor: 0.1
    missing_value: 32767
    units: mm/day
unlimited dimensions: time
current shape = (16801, 17, 16)
filling 

In [8]:
data = dataset.variables['data'][:]

In [9]:
data

masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., 6.800000190734863, 10.40000057220459, --],
         ...,
         [--, --, 27.399999618530273, ..., 1.100000023841858, 1.5, --],
         [--, --, 31.0, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., 0.0, 0.0, --],
         ...,
         [--, --, 21.399999618530273, ..., 0.0, 0.0, --],
         [--, --, 4.0, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., 0.0, 0.0, --],
         ...,
         [--, --, 0.0, ..., 0.0, 0.0, --],
         [--, --, 0.0, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        ...,

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., 0.0, 0

In [10]:
# Flatten the data array to 2D
data_flat = data.reshape(data.shape[0], -1)

In [11]:
time = dataset.variables['time'][:]


In [12]:
import numpy as np

In [13]:
years = np.unique(time.astype(int))  # Get unique years
annual_precipitation = []
for year in years:
    year_indices = np.where(time.astype(int) == year)[0]
    year_precipitation = np.sum(data[year_indices, :, :])
    annual_precipitation.append(year_precipitation)

In [14]:
# Prepare the feature matrix (X) and target variable (y)
X = years.reshape(-1, 1)
y = np.array(annual_precipitation)

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [17]:
# Make predictions
y_pred = model.predict(X_test)

In [18]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [19]:
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 477061.8860191916
R^2 Score: -9.657225465886654e-05


In [20]:
# Close the dataset
dataset.close()